# Übungsblatt 8

**Lernziele**

In den Übungen dieser Wochen lernen Sie:

* Funktionen und Assoziative Listen zur Darstellung von Abbildungen (Maps) einzusetzen.
* Anonyme Funktionen als Werte und Argumente für andere Funktionen einzusetzen.
* Operationen auf Listen mithilfe von fold_left zu implementieren.
* Matrizen mit verschiedenen Typ-Konstrukten zu modellieren.
* Verschiedene Graph-Algorithmen in funktionalen Sprachen zu implementieren.

## Aufgabe 8.1 (P) Abbildungen

Obwohl Listen den Umgang mit einer sequentiellen (möglicherweise sortierten) Folge von
Werten (relativ) einfach machen, benötigen größere Anwendungen oft auch Datenstrukturen,
die andere Eigenschaften haben und/oder bestimmte Operationen effizienter durchführen
können. Eine Datenstruktur, die besonders häufig zum Einsatz kommt, ist dabei
die `Map`. Sie bildet Schlüssel auf Werte ab.

1\. In der letzten Woche haben Sie bereits eine Möglichkeit kennengelernt, wie eine
solche Abbildung in funktionalen Sprachen realisiert werden kann. Beschreiben Sie
diese und gehen Sie dabei besonders darauf ein, wie nach Schlüsseln gesucht, bzw.
wie Einträge hinzugefügt und entfernt werden können.

In [1]:
let map = function "a" -> 5 | "b" -> 7 | "c" -> 19

File "[1]", line 1, characters 10-50:
Warning 8: this pattern-matching is not exhaustive.
Here is an example of a case that is not matched:


val map : string -> int = <fun>


In [2]:
map "a"

""


- : int = 5


In [3]:
map "d"

error: error

Ein neues Element kann in die Map eingefügt werden, indem eine neue Funktion definiert wird, die für bestimmte Werte Konstanten zurückgibt und sonst die ursprüngliche Map aufruft.

In [4]:
let map x = if x = "d" then 3 else map x

val map : string -> int = <fun>


In [5]:
map "d"

- : int = 3


Ein Element wird wieder gelöscht, indem eine Exception geworfen wird, falls dieses Element angefragt wird.

In [6]:
let map x = if x = "d" then raise (Match_failure ("", 0, 0)) else map x

val map : string -> int = <fun>


In [7]:
map "d"

error: error

2\. Eine Alternative um Schlüssel-Wert-Paare darzustellen sind Listen, die eben entsprechende
Paare speichern (`('a * 'b) list`). Diese Art von Listen wird üblicherweise
als *Assoziative Liste* bezeichnet. Überlegen Sie sich zunächst, welchen Typen die folgenden
Funktionen auf assoziativien Listen haben müssen, dann recherchieren Sie,
welche Standardfunktionen das OCaml `List`-Modul für die diese Operationen zur
Verfügung stellt:

* Prüfen, ob ein Schlüssel in der Liste existiert.
* Den Wert zu einem Schlüssel in der Liste finden.
* Einen Schlüssel (und dessen Wert) aus der Liste entfernen.
* Alle Schlüssel der Liste abfragen.

[OCaml List manual](http://caml.inria.fr/pub/docs/manual-ocaml/libref/List.html)

In [8]:
let map = [("a", 5); ("b", 7); ("c", 19)]

val map : (string * int) list = [("a", 5); ("b", 7); ("c", 19)]


Prüfen, ob ein Schlüssel in der Liste existiert.

In [9]:
List.mem_assoc

- : 'a -> ('a * 'b) list -> bool = <fun>


In [10]:
List.mem_assoc "a" map

- : bool = true


In [11]:
List.mem_assoc "d" map

- : bool = false


Den Wert zu einem Schlüssel in der Liste finden.

In [12]:
List.assoc

- : 'a -> ('a * 'b) list -> 'b = <fun>


In [13]:
List.assoc "a" map

- : int = 5


In [14]:
List.assoc "d" map

error: error

Einen Schlüssel (und dessen Wert) aus der Liste entfernen.

In [15]:
List.remove_assoc

- : 'a -> ('a * 'b) list -> ('a * 'b) list = <fun>


In [16]:
List.remove_assoc "a" map

- : (string * int) list = [("b", 7); ("c", 19)]


In [17]:
List.remove_assoc "d" map

- : (string * int) list = [("a", 5); ("b", 7); ("c", 19)]


Alle Schlüssel der Liste abfragen.

In [18]:
List.split

- : ('a * 'b) list -> 'a list * 'b list = <fun>


In [19]:
List.split map

- : string list * int list = (["a"; "b"; "c"], [5; 7; 19])


3\. Vergleichen Sie die beiden Alternativen zur Darstellung von Abbildungen bezüglich
Usability und Performance.

* Abfragen von Werten bei beiden Varianten sehr einfach
* Listen bieten zusätzliche Funktionalität, z.B. die Liste aller Schlüssel und Werte
* In einer Liste kann ein Element nur durch linears Durchuchen gefunden werden, daher können Funktionen effizienter sein
* Noch besser ist eine Suchstruktur (z.B. Rot-Schwarz-Baum oder B-Baum) oder Hashing

## Aufgabe 8.2 (P) Matrix Reloaded

Ein Vektor und eine Matrix seien durch folgende Typen definiert:

In [20]:
type vector = float list
type matrix = float list list

type vector = float list


type matrix = float list list


In [21]:
let v = [1.;2.;3.]

val v : float list = [1.; 2.; 3.]


In [22]:
let m = [[1.;2.;3.];
         [4.;5.;6.];
         [7.;8.;9.]]

val m : float list list = [[1.; 2.; 3.]; [4.; 5.; 6.]; [7.; 8.; 9.]]


1. Diskutieren Sie die Vor- und Nachteile dieser Darstellung, sowie mögliche Alternativen.
2. Implementieren Sie die folgenden Funktionen mit Hilfe der Listenfunktionen der Standardbibliothek. Sie dürfen keine eigenen rekursiven Funktionen definieren:
    * `multiplys : matrix -> float -> matrix`, die eine Matrix mit einem Skalar multipliziert.
    * `multiplyv : matrix -> vector -> vector`, die eine Matrix mit einem Vektor multipliziert.
    * `transpose : matrix -> matrix`, die eine Matrix transponiert.
    * `multiplym : matrix -> matrix -> matrix`, die zwei Matrizen multipliziert.

**Matrix-Skalar Produkt**

$$
    \begin{pmatrix}
        a_{11} & \cdots & a_{1n} \\
        \vdots & \ddots & \vdots \\
        a_{m1} & \cdots & a_{mn} \\
    \end{pmatrix}
    \cdot s =
    \begin{pmatrix}
        a_{11} \cdot s & \cdots & a_{1n} \cdot s \\
        \vdots & \ddots & \vdots \\
        a_{m1} \cdot s & \cdots & a_{mn} \cdot s \\
    \end{pmatrix}
$$

In [23]:
let multiplys m s = List.map (List.map (fun a -> a *. s)) m

val multiplys : float list list -> float -> float list list = <fun>


In [24]:
multiplys m 3.

- : float list list = [[3.; 6.; 9.]; [12.; 15.; 18.]; [21.; 24.; 27.]]


**Matrix-Vektor Produkt**

$$
    \begin{pmatrix}
        a_{11} & \cdots & a_{1n} \\
        \vdots & \ddots & \vdots \\
        a_{m1} & \cdots & a_{mn} \\
    \end{pmatrix}
    \cdot
    \begin{pmatrix}
        v_1 \\ \vdots \\ v_n
    \end{pmatrix}
    =
    \begin{pmatrix}
        a_{11} \cdot v_1 + \cdots + a_{1n} \cdot v_n \\
        \vdots \\
        a_{m1} \cdot v_1 + \cdots + a_{mn} \cdot v_n \\
    \end{pmatrix}
$$

In [25]:
List.fold_left2

- : ('a -> 'b -> 'c -> 'a) -> 'a -> 'b list -> 'c list -> 'a = <fun>


In [26]:
let multiplyv m v = List.map (List.fold_left2 (fun z v a -> z +. v *. a) 0. v) m

val multiplyv : float list list -> float list -> float list = <fun>


In [27]:
multiplyv m v

- : float list = [14.; 32.; 50.]


**Transposition**

$$
    \begin{pmatrix}
        a_{11} & \cdots & a_{1n} \\
        \vdots & \ddots & \vdots \\
        a_{m1} & \cdots & a_{mn} \\
    \end{pmatrix}^T =
    \begin{pmatrix}
        a_{11} & \cdots & a_{m1} \\
        \vdots & \ddots & \vdots \\
        a_{1n} & \cdots & a_{mn} \\
    \end{pmatrix}
$$

In [28]:
List.fold_right

- : ('a -> 'b -> 'b) -> 'a list -> 'b -> 'b = <fun>


In [29]:
List.hd

- : 'a list -> 'a = <fun>


In [30]:
List.map2

- : ('a -> 'b -> 'c) -> 'a list -> 'b list -> 'c list = <fun>


In [31]:
let transpose m =
  let e = List.map (fun _ -> []) (List.hd m) in
  List.fold_right (List.map2 (fun a r -> a :: r)) m e

val transpose : 'a list list -> 'a list list = <fun>


In [32]:
transpose m

- : float list list = [[1.; 4.; 7.]; [2.; 5.; 8.]; [3.; 6.; 9.]]


In [33]:
transpose [[1.];[2.];[3.]]

- : float list list = [[1.; 2.; 3.]]


**Matrix-Matrix Produkt**

$$
    \begin{pmatrix}
        a_{11} & \cdots & a_{1m} \\
        \vdots & \ddots & \vdots \\
        a_{l1} & \cdots & a_{lm} \\
    \end{pmatrix}
    \cdot
    \begin{pmatrix}
        b_{11} & \cdots & b_{1n} \\
        \vdots & \ddots & \vdots \\
        b_{m1} & \cdots & b_{mn} \\
    \end{pmatrix}
    =
    \begin{pmatrix}
        a_{11} \cdot b_{11} + \cdots + a_{1m} \cdot b_{m1} & \cdots &
        a_{11} \cdot b_{1n} + \cdots + a_{1m} \cdot b_{mn}
        \\
        \vdots & \ddots & \vdots \\
        a_{l1} \cdot b_{11} + \cdots + a_{lm} \cdot b_{m1} & \cdots &
        a_{l1} \cdot b_{1n} + \cdots + a_{lm} \cdot b_{mn}
        \\
    \end{pmatrix}
$$

In [34]:
let multiplym m1 m2 = transpose (List.map (multiplyv m1) (transpose m2))

val multiplym : float list list -> float list list -> float list list = <fun>


Es gilt $ (AB)^T = B^TA^T $ und damit können wir noch vereinfachen.

In [35]:
let multiplym m1 m2 = List.map (multiplyv (transpose m2)) m1

val multiplym : float list list -> float list list -> float list list = <fun>


In [36]:
let m1 = [[1.;2.];
          [3.;4.];
          [5.;6.]]

let m2 = [[1.;2.;3.];
          [4.;5.;6.]]

val m1 : float list list = [[1.; 2.]; [3.; 4.]; [5.; 6.]]


val m2 : float list list = [[1.; 2.; 3.]; [4.; 5.; 6.]]


In [37]:
multiplym m1 m2

- : float list list = [[9.; 12.; 15.]; [19.; 26.; 33.]; [29.; 40.; 51.]]


In [38]:
multiplyv m1 [1.;4.]

- : float list = [9.; 19.; 29.]


In [39]:
multiplyv m1 [2.;5.]

- : float list = [12.; 26.; 40.]


In [40]:
multiplyv m1 [3.;6.]

- : float list = [15.; 33.; 51.]


## Aufgabe 8.3 (P) Unendlich faule Listen

Neben den Standard OCaml Listen, haben Sie bisher das Konzept nicht-leerer Listen
kennengelernt. Im Folgenden werden Sie eine weitere Limitierung dieser Listen aufheben:
die Beschränkung auf endliche Längen.

1\. Diskutieren Sie, welche Grundvoraussetzungen gegeben sein müssen, damit eine unendliche Liste erzeugt werden kann.

Wir brauchen eine Möglichkeit, Ausdrücke erst auszuwerten, wenn wir sie wirklich brauchen, da wir nicht die gesamte unendliche Liste im Hauptspeicher instantiieren können. Eine solche Auswertung der Ausdrücke bezeichnet man als *lazy*.

In OCaml lässt sich dies mit Funktionen realisieren, deren Inhalt erst ausgewertet wird, wenn sie aufgerufen werden. Als Konvention wird als Parametertyp meist `unit` verwendet. Soll beispielsweise die komplexe Operation `3 * 4` nicht sofort ausgeführt werden, sondern erst, wenn das Ergebnis benötigt wird, so kann man stattdessen die Funktion `fun () -> 3 * 4` definieren, die die Berechnung ausführt, wenn sie mit `()` als Parameter aufgerufen wird.

In [41]:
3 * 4

- : int = 12


In [42]:
let f = fun () -> 3 * 4

val f : unit -> int = <fun>


In [43]:
f ()

- : int = 12


2\. Definieren Sie einen Typen `'a llist`, der unendliche Listen darstellt.

In [44]:
type 'a llist = Cons of 'a * (unit -> 'a llist)

type 'a llist = Cons of 'a * (unit -> 'a llist)


3\. Implementieren Sie die folgenden Funktionen, die entsprechende unendliche Listen erzeugen:

(a) `lconst : int -> int llist` erzeugt eine konstante Folge.

In [45]:
let rec lconst n = Cons (n, fun () -> lconst n)

val lconst : 'a -> 'a llist = <fun>


In [46]:
let Cons (h, t) = lconst 5

val h : int = 5
val t : unit -> int llist = <fun>


In [47]:
let Cons (h, t) = t ()

val h : int = 5
val t : unit -> int llist = <fun>


(b) `lseq : int -> int llist` erzeugt eine aufsteigende Folge, beginnend mit
dem übergebenen Element.


In [48]:
let rec lseq n = Cons (n, fun () -> lseq (n + 1))

val lseq : int -> int llist = <fun>


In [49]:
let Cons (h, t) = lseq 5

val h : int = 5
val t : unit -> int llist = <fun>


In [50]:
let Cons (h, t) = t ()

val h : int = 6
val t : unit -> int llist = <fun>


(c) `lpowers2 : unit -> int llist` erzeugt eine Folge von 2er-Potenzen beginnend
mit 1.


In [51]:
let lpowers2 () =
  let rec impl n = Cons (n, fun () -> impl (2 * n))
  in impl 1

val lpowers2 : unit -> int llist = <fun>


In [52]:
let Cons (h, t) = lpowers2 ()

val h : int = 1
val t : unit -> int llist = <fun>


In [53]:
let Cons (h, t) = t ()

val h : int = 2
val t : unit -> int llist = <fun>


In [54]:
let Cons (h, t) = t ()

val h : int = 4
val t : unit -> int llist = <fun>


(d) `lfib : unit -> int llist` erzeugt die Fibonacci-Folge (beginnend mit 0
und 1).

In [55]:
let lfib () =
  let rec impl a b = Cons (a, fun () -> impl b (a + b))
  in impl 0 1

val lfib : unit -> int llist = <fun>


In [56]:
let Cons (h, t) = lfib ()

val h : int = 0
val t : unit -> int llist = <fun>


In [57]:
let Cons (h, t) = t ()

val h : int = 1
val t : unit -> int llist = <fun>


4\. Entscheiden Sie für die folgenden Funktionen, ob diese sinnvoll für unendliche Listen
implementiert werden können. Implementieren Sie die Funktion, wenn dies der Fall
ist:


(a) `lhd : 'a llist -> 'a` liefert den Kopf der Liste.


In [58]:
let lhd (Cons (h, t)) = h

val lhd : 'a llist -> 'a = <fun>


In [59]:
lhd (lseq 5)

- : int = 5


(b) `ltl : 'a llist -> 'a llist` liefert den Schwanz der Liste.


In [60]:
let ltl (Cons (h, t)) = t ()

val ltl : 'a llist -> 'a llist = <fun>


In [61]:
ltl (lseq 5)

- : int llist = Cons (6, <fun>)


(c) `ltake : int -> 'a llist -> 'a list` gibt die ersten n (1. Argument) Elemente
der Liste zurück.


In [62]:
let rec ltake n (Cons (h, t)) =
  if n <= 0 then
    []
  else
    h :: ltake (n - 1) (t ())

val ltake : int -> 'a llist -> 'a list = <fun>


In [63]:
ltake 10 (lconst 5)

- : int list = [5; 5; 5; 5; 5; 5; 5; 5; 5; 5]


In [64]:
ltake 10 (lseq 5)

- : int list = [5; 6; 7; 8; 9; 10; 11; 12; 13; 14]


In [65]:
ltake 10 (lpowers2 ())

- : int list = [1; 2; 4; 8; 16; 32; 64; 128; 256; 512]


In [66]:
ltake 10 (lfib ())

- : int list = [0; 1; 1; 2; 3; 5; 8; 13; 21; 34]


(d) `ldrop : int -> 'a llist -> 'a llist` entfernt die ersten n (1. Argument)
Elemente der Liste.


In [67]:
let rec ldrop n (Cons (h, t)) =
  if n <= 0 then
    Cons (h, t)
  else
    ldrop (n - 1) (t ())

val ldrop : int -> 'a llist -> 'a llist = <fun>


In [68]:
let l = ldrop 10 (lseq 5)

val l : int llist = Cons (15, <fun>)


In [69]:
ltake 10 l

- : int list = [15; 16; 17; 18; 19; 20; 21; 22; 23; 24]


(e) `lappend : 'a llist -> 'a llist -> 'a llist` hängt zwei Listen aneinander.


In [70]:
(* nicht sinnvoll *)

(f) `lreverse : 'a llist -> 'a llist` dreht die Liste um.


In [71]:
(* nicht sinnvoll *)

(g) `lfilter : ('a -> bool) -> 'a llist -> 'a llist` filtert die Elemente der
Liste mit einem Prädikat (1. Argument).


In [72]:
let rec lfilter f (Cons (h, t)) =
  if f h then
    Cons (h, fun () -> lfilter f (t ()))
  else
    lfilter f (t ())

val lfilter : ('a -> bool) -> 'a llist -> 'a llist = <fun>


In [73]:
let l = lfilter (fun x -> x mod 2 = 0) (lseq 0)

val l : int llist = Cons (0, <fun>)


In [74]:
ltake 10 l

- : int list = [0; 2; 4; 6; 8; 10; 12; 14; 16; 18]


In [75]:
(* Problematisch:
lfilter (fun _ -> false) (lconst 0)
*)

(h) `lmap : ('a -> 'b) -> 'a llist -> 'b llist` transformiert alle Elemente
der Liste mit der gegebenen Funktion (1. Argument).

In [76]:
let rec lmap f (Cons (h, t)) = Cons (f h, fun () -> lmap f (t ()))

val lmap : ('a -> 'b) -> 'a llist -> 'b llist = <fun>


In [77]:
let l = lmap (fun x -> x * 3) (lseq 0)

val l : int llist = Cons (0, <fun>)


In [78]:
ltake 10 l

- : int list = [0; 3; 6; 9; 12; 15; 18; 21; 24; 27]
